In [0]:
%sh
ls -la /home/finetuned_lm_eng/data

In [0]:
from argparse import ArgumentParser
from pathlib import Path
import os
import torch
import logging
import json
import random
import numpy as np 
from collections import namedtuple
from tempfile import TemporaryDirectory

import logging
logger = spark._jvm.org.apache.log4j
logging.getLogger("py4j").setLevel(logging.ERROR)
from torch.utils.data import DataLoader, Dataset, RandomSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm

from pytorch_transformers import WEIGHTS_NAME, CONFIG_NAME
from pytorch_transformers.modeling_bert import BertForPreTraining
from pytorch_transformers.tokenization_bert import BertTokenizer
from pytorch_transformers.optimization import AdamW, WarmupLinearSchedule

InputFeatures = namedtuple("InputFeatures", "input_ids input_mask segment_ids lm_label_ids is_next")

log_format = '%(asctime)-10s: %(message)s'
logging.basicConfig(level=logging.INFO, format=log_format)


def convert_example_to_features(example, tokenizer, max_seq_length):
    tokens = example["tokens"]
    segment_ids = example["segment_ids"]
    is_random_next = example["is_random_next"]
    masked_lm_positions = example["masked_lm_positions"]
    masked_lm_labels = example["masked_lm_labels"]

    assert len(tokens) == len(segment_ids) <= max_seq_length  # The preprocessed data should be already truncated
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    masked_label_ids = tokenizer.convert_tokens_to_ids(masked_lm_labels)

    input_array = np.zeros(max_seq_length, dtype=np.int)
    input_array[:len(input_ids)] = input_ids

    mask_array = np.zeros(max_seq_length, dtype=np.bool)
    mask_array[:len(input_ids)] = 1

    segment_array = np.zeros(max_seq_length, dtype=np.bool)
    segment_array[:len(segment_ids)] = segment_ids

    lm_label_array = np.full(max_seq_length, dtype=np.int, fill_value=-1)
    lm_label_array[masked_lm_positions] = masked_label_ids

    features = InputFeatures(input_ids=input_array,
                             input_mask=mask_array,
                             segment_ids=segment_array,
                             lm_label_ids=lm_label_array,
                             is_next=is_random_next)
    return features


class PregeneratedDataset(Dataset):
    def __init__(self, training_path, epoch, tokenizer, num_data_epochs, reduce_memory=False):
        self.vocab = tokenizer.vocab
        self.tokenizer = tokenizer
        self.epoch = epoch
        self.data_epoch = epoch % num_data_epochs
        data_file = training_path / f"epoch_{self.data_epoch}.json"
        metrics_file = training_path / f"epoch_{self.data_epoch}_metrics.json"
        assert data_file.is_file() and metrics_file.is_file()
        metrics = json.loads(metrics_file.read_text())
        num_samples = metrics['num_training_examples']
        seq_len = metrics['max_seq_len']
        self.temp_dir = None
        self.working_dir = None
        if reduce_memory:
            self.temp_dir = TemporaryDirectory()
            self.working_dir = Path(self.temp_dir.name)
            input_ids = np.memmap(filename=self.working_dir/'input_ids.memmap',
                                  mode='w+', dtype=np.int32, shape=(num_samples, seq_len))
            input_masks = np.memmap(filename=self.working_dir/'input_masks.memmap',
                                    shape=(num_samples, seq_len), mode='w+', dtype=np.bool)
            segment_ids = np.memmap(filename=self.working_dir/'segment_ids.memmap',
                                    shape=(num_samples, seq_len), mode='w+', dtype=np.bool)
            lm_label_ids = np.memmap(filename=self.working_dir/'lm_label_ids.memmap',
                                     shape=(num_samples, seq_len), mode='w+', dtype=np.int32)
            lm_label_ids[:] = -1
            is_nexts = np.memmap(filename=self.working_dir/'is_nexts.memmap',
                                 shape=(num_samples,), mode='w+', dtype=np.bool)
        else:
            input_ids = np.zeros(shape=(num_samples, seq_len), dtype=np.int32)
            input_masks = np.zeros(shape=(num_samples, seq_len), dtype=np.bool)
            segment_ids = np.zeros(shape=(num_samples, seq_len), dtype=np.bool)
            lm_label_ids = np.full(shape=(num_samples, seq_len), dtype=np.int32, fill_value=-1)
            is_nexts = np.zeros(shape=(num_samples,), dtype=np.bool)
        logging.info(f"Loading training examples for epoch {epoch}")
        with data_file.open() as f:
            for i, line in enumerate(tqdm(f, total=num_samples, desc="Training examples")):
                line = line.strip()
                example = json.loads(line)
                features = convert_example_to_features(example, tokenizer, seq_len)
                input_ids[i] = features.input_ids
                segment_ids[i] = features.segment_ids
                input_masks[i] = features.input_mask
                lm_label_ids[i] = features.lm_label_ids
                is_nexts[i] = features.is_next
        assert i == num_samples - 1  # Assert that the sample count metric was true
        logging.info("Loading complete!")
        self.num_samples = num_samples
        self.seq_len = seq_len
        self.input_ids = input_ids
        self.input_masks = input_masks
        self.segment_ids = segment_ids
        self.lm_label_ids = lm_label_ids
        self.is_nexts = is_nexts

    def __len__(self):
        return self.num_samples

    def __getitem__(self, item):
        return (torch.tensor(self.input_ids[item].astype(np.int64)),
                torch.tensor(self.input_masks[item].astype(np.int64)),
                torch.tensor(self.segment_ids[item].astype(np.int64)),
                torch.tensor(self.lm_label_ids[item].astype(np.int64)),
                torch.tensor(self.is_nexts[item].astype(np.int64)))


def main():
    parser = ArgumentParser()
    parser.add_argument('--pregenerated_data', type=Path, required=True)
    parser.add_argument('--output_dir', type=Path, required=True)
    parser.add_argument("--bert_model", type=str, required=True, help="Bert pre-trained model selected in the list: bert-base-uncased, "
                             "bert-large-uncased, bert-base-cased, bert-base-multilingual, bert-base-chinese.")
    
    
    #pregenerated_data_path =  'finetuned_lm_3e/data/'
    pregenerated_data_path =  'finetuned_lm_eng/data/'
    pregenerated_data = Path(pregenerated_data_path)
    #train_corpus = '/tables/corpus.txt'
    output_dir_path = '/home/finetune_lm/model_eng/'
    output_dir = Path(output_dir_path)
    #bert_model = 'bert-base-multilingual-cased'
    bert_model = 'bert-base-cased'
    
    do_lower_case = False
    reduce_memory = False
    epochs = 3
    local_rank = -1
    no_cuda = True
    gradient_accumulation_steps = 1
    train_batch_size = 32
    fp16 = False
    loss_scale = 0.0
    warmup_steps = 0
    adam_epsilon = 1e-8
    learning_rate = 3e-5
    seed = 42

    assert pregenerated_data.is_dir(), \
        "--pregenerated_data should point to the folder of files made by pregenerate_training_data.py!"

    samples_per_epoch = []
    for i in range(epochs):
        epoch_file = pregenerated_data / f"epoch_{i}.json"
        metrics_file = pregenerated_data / f"epoch_{i}_metrics.json"
        if epoch_file.is_file() and metrics_file.is_file():
            metrics = json.loads(metrics_file.read_text())
            samples_per_epoch.append(metrics['num_training_examples'])
        else:
            if i == 0:
                exit("No training data was found!")
            print(f"Warning! There are fewer epochs of pregenerated data ({i}) than training epochs ({epochs}).")
            print("This script will loop over the available data, but training diversity may be negatively impacted.")
            num_data_epochs = i
            break
    else:
        num_data_epochs = epochs

    if local_rank == -1 or no_cuda:
        device = torch.device("cuda" if torch.cuda.is_available() and not no_cuda else "cpu")
        n_gpu = torch.cuda.device_count()
    else:
        torch.cuda.set_device(local_rank)
        device = torch.device("cuda", local_rank)
        n_gpu = 1
        # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
        torch.distributed.init_process_group(backend='nccl')
    logging.info("device: {} n_gpu: {}, distributed training: {}, 16-bits training: {}".format(
        device, n_gpu, bool(local_rank != -1), fp16))

    if gradient_accumulation_steps < 1:
        raise ValueError("Invalid gradient_accumulation_steps parameter: {}, should be >= 1".format(
                            gradient_accumulation_steps))

    train_batch_size = train_batch_size // gradient_accumulation_steps

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if n_gpu > 0:
        torch.cuda.manual_seed_all(seed)

    if output_dir.is_dir() and list(output_dir.iterdir()):
        logging.warning(f"Output directory ({output_dir}) already exists and is not empty!")
    output_dir.mkdir(parents=True, exist_ok=True)

    tokenizer = BertTokenizer.from_pretrained(bert_model, do_lower_case=do_lower_case)

    total_train_examples = 0
    for i in range(epochs):
        # The modulo takes into account the fact that we may loop over limited epochs of data
        total_train_examples += samples_per_epoch[i % len(samples_per_epoch)]

    num_train_optimization_steps = int(
        total_train_examples / train_batch_size / gradient_accumulation_steps)
    if local_rank != -1:
        num_train_optimization_steps = num_train_optimization_steps // torch.distributed.get_world_size()

    # Prepare model
    model = BertForPreTraining.from_pretrained(bert_model)
    # We don't need to manually call model.half() following Apex's recommend
    # if fp16:
    #     model.half()
    model.to(device)
    if local_rank != -1:
        try:
            from apex.parallel import DistributedDataParallel as DDP
        except ImportError:
            raise ImportError(
                "Please install apex from https://www.github.com/nvidia/apex to use distributed and fp16 training.")
        model = DDP(model)
    elif n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Prepare optimizer
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

    optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)
    scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_steps,
                                     t_total=num_train_optimization_steps)

    if fp16:
        try:
            # from apex.optimizers import FP16_Optimizer
            # from apex.optimizers import FusedAdam
            from apex import amp
        except ImportError:
            raise ImportError(
                "Please install apex from https://www.github.com/nvidia/apex to use distributed and fp16 training.")

        # This below line of code is the main upgrade of Apex Fp16 implementation. I chose opt_leve="01"
        # because it's recommended for typical use by Apex. We can make it configured
        model, optimizer = amp.initialize(model, optimizer, opt_level="O1")

    # We don't need to use FP16_Optimizer wrapping over FusedAdam as well. Now Apex supports all Pytorch Optimizer

    #     optimizer = FusedAdam(optimizer_grouped_parameters,
    #                           lr=learning_rate,
    #                           bias_correction=False,
    #                           max_grad_norm=1.0)
    #     if loss_scale == 0:
    #         optimizer = FP16_Optimizer(optimizer, dynamic_loss_scale=True)
    #     else:
    #         optimizer = FP16_Optimizer(optimizer, static_loss_scale=loss_scale)
    # else:
    #     optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)
    # scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_steps, t_total=num_train_optimization_steps)

    global_step = 0
    logging.info("***** Running training *****")
    logging.info(f"  Num examples = {total_train_examples}")
    logging.info("  Batch size = %d", train_batch_size)
    logging.info("  Num steps = %d", num_train_optimization_steps)
    model.train()
    for epoch in range(epochs):
        epoch_dataset = PregeneratedDataset(epoch=epoch, training_path=pregenerated_data, tokenizer=tokenizer,
                                            num_data_epochs=num_data_epochs, reduce_memory=reduce_memory)
        if local_rank == -1:
            train_sampler = RandomSampler(epoch_dataset)
        else:
            train_sampler = DistributedSampler(epoch_dataset)
        train_dataloader = DataLoader(epoch_dataset, sampler=train_sampler, batch_size=train_batch_size)
        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0
        with tqdm(total=len(train_dataloader), desc=f"Epoch {epoch}") as pbar:
            for step, batch in enumerate(train_dataloader):
                batch = tuple(t.to(device) for t in batch)
                input_ids, input_mask, segment_ids, lm_label_ids, is_next = batch
                outputs = model(input_ids, segment_ids, input_mask, lm_label_ids, is_next)
                loss = outputs[0]
                if n_gpu > 1:
                    loss = loss.mean() # mean() to average on multi-gpu.
                if gradient_accumulation_steps > 1:
                    loss = loss / gradient_accumulation_steps
                if fp16:
                    # I depricate FP16_Optimizer's backward func and replace as Apex document
                    # optimizer.backward(loss)
                    with amp.scale_loss(loss, optimizer) as scaled_loss:
                        scaled_loss.backward()
                else:
                    loss.backward()
                tr_loss += loss.item()
                nb_tr_examples += input_ids.size(0)
                nb_tr_steps += 1
                pbar.update(1)
                mean_loss = tr_loss * gradient_accumulation_steps / nb_tr_steps
                pbar.set_postfix_str(f"Loss: {mean_loss:.5f}")
                if (step + 1) % gradient_accumulation_steps == 0:
                    optimizer.step()
                    scheduler.step()  # Update learning rate schedule
                    optimizer.zero_grad()
                    global_step += 1

    # Save a trained model
    if local_rank == -1 or torch.distributed.get_rank() == 0:
        logging.info("** ** * Saving fine-tuned model ** ** * ")
        model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
        model_to_save.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)


if __name__ == '__main__':
    main()

2019-09-13 09:17:28,494: device: cpu n_gpu: 0, distributed training: False, 16-bits training: False
2019-09-13 09:17:29,003: https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt not found in cache, downloading to /tmp/tmp1l4zc8ab
 0%| | 0/213450 [00:00<?, ?B/s] 8%|▊ | 17408/213450 [00:00<00:01, 105573.23B/s] 24%|██▍ | 52224/213450 [00:00<00:01, 123878.00B/s] 49%|████▉ | 104448/213450 [00:00<00:00, 151197.16B/s] 82%|████████▏ | 174080/213450 [00:00<00:00, 186890.14B/s]100%|██████████| 213450/213450 [00:00<00:00, 318858.23B/s]
2019-09-13 09:17:30,044: copying /tmp/tmp1l4zc8ab to cache at /root/.cache/torch/pytorch_transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
2019-09-13 09:17:30,045: creating metadata file for /root/.cache/torch/pytorch_transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
2019-09-13 09:17:30,046: removing temp file /tmp/tmp1l4zc8ab
2019-09-13 09:17:30,046: loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at /root/.cache/torch/pytorch_transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
2019-09-13 09:17:30,508: https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json not found in cache, downloading to /tmp/tmp64tzlt8_
 0%| | 0/313 [00:00<?, ?B/s]100%|██████████| 313/313 [00:00<00:00, 251835.25B/s]
2019-09-13 09:17:30,903: copying /tmp/tmp64tzlt8_ to cache at /root/.cache/torch/pytorch_transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.d7a3af18ce3a2ab7c0f48f04dc8daff45ed9a3ed333b9e9a79d012a0dedf87a6
2019-09-13 09:17:30,903: creating metadata file for /root/.cache/torch/pytorch_transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.d7a3af18ce3a2ab7c0f48f04dc8daff45ed9a3ed333b9e9a79d012a0dedf87a6
2019-09-13 09:17:30,903: removing temp file /tmp/tmp64tzlt8_
2019-09-13 09:17:30,904: loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json from cache at /root/.cache/torch/pytorch_transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.d7a3af18ce3a2ab7c0f48f04dc8daff45ed9a3ed333b9e9a79d012a0dedf87a6
2019-09-13 09:17:30,904: Model config {
 "attention_probs_dropout_prob": 0.1,
 "finetuning_task": null,
 "hidden_act": "gelu",
 "hidden_dropout_prob": 0.1,
 "hidden_size": 768,
 "initializer_range": 0.02,
 "intermediate_size": 3072,
 "layer_norm_eps": 1e-12,
 "max_position_embeddings": 512,
 "num_attention_heads": 12,
 "num_hidden_layers": 12,
 "num_labels": 2,
 "output_attentions": false,
 "output_hidden_states": false,
 "torchscript": false,
 "type_vocab_size": 2,
 "vocab_size": 28996
}

2019-09-13 09:17:31,300: https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-pytorch_model.bin not found in cache, downloading to /tmp/tmpe8rv61u3
 0%| | 0/435779157 [00:00<?, ?B/s] 0%| | 52224/435779157 [00:00<22:59, 315795.49B/s] 0%| | 278528/435779157 [00:00<17:41, 410271.85B/s] 0%| | 957440/435779157 [00:00<12:53, 561864.09B/s] 1%| | 2353152/435779157 [00:00<09:09, 789046.61B/s] 1%| | 4104192/435779157 [00:00<06:34, 1095466.28B/s] 2%|▏ | 6861824/435779157 [00:00<04:38, 1538752.24B/s] 2%|▏ | 9433088/435779157 [00:00<03:18, 2143248.42B/s] 3%|▎ | 11591680/435779157 [00:01<02:26, 2897650.70B/s] 3%|▎ | 14273536/435779157 [00:01<01:46, 3956297.94B/s] 4%|▍ | 16934912/435779157 [00:01<01:18, 5313314.66B/s] 4%|▍ | 19119104/435779157 [00:01<01:00, 6859928.93B/s] 5%|▌ | 22667264/435779157 [00:01<00:46, 8953528.18B/s] 6%|▌ | 26138624/435779157 [00:01<00:35, 11517550.03B/s] 7%|▋ | 28897280/435779157 [00:01<00:29, 13663832.32B/s] 7%|▋ | 32110592/435779157 [00:01<00:24, 16507012.82B/s] 8%|▊ | 34988032/435779157 [00:01<0

In [0]:
%sh
echo hello
ls -la /home/
ls -la /home/finetune_lm/
ls -la /home/finetune_lm/model_eng/

hello
total 4
drwxrwxrwx 1 root root 4096 Jan 1 1970 .
drwxr-xr-x 1 root root 4096 Sep 13 07:30 ..
drwxr-xr-x 1 root root 0 May 31 13:58 ##########
drwxr-xr-x 1 root root 0 Apr 24 15:57 C
drwxr-xr-x 1 root root 0 Jan 1 1970 home-datasets
drwxr-xr-x 1 root root 0 Jun 12 15:07 home-results
drwxr-xr-x 1 root root 0 Jun 21 13:37 home
drwxr-xr-x 1 root root 0 Apr 24 15:56 file
drwxr-xr-x 1 root root 0 Mar 13 2019 FileStore
drwxr-xr-x 1 root root 0 Sep 13 08:33 finetune_lm
drwxr-xr-x 1 root root 0 Apr 24 14:00 list_errors.csv
drwxr-xr-x 1 root root 0 Apr 24 14:54 list_errors_robin.csv
drwxrwx--- 2 root root 4096 Jan 1 1970 ml
drwxr-xr-x 1 root root 0 Mar 11 2019 mnt
drwxr-xr-x 1 root root 0 Apr 17 11:45 mycsv.csv
drwxr-xr-x 1 root root 0 Jun 21 14:38 TestGPA
drwxr-xr-x 1 root root 0 Mar 4 2019 tmp
drwxr-xr-x 1 root root 0 Apr 16 08:37 user
drwxr-xr-x 1 root root 0 Aug 7 07:33 vgadlvceaadevsa01.blob.core.windows.net
total 0
drwxr-xr-x 1 root root 0 Sep 13 08:33 .
drwxrwxrwx 1 root root 4096 Jan 1 1970 ..
drwxr-xr-x 1 root root 0 Sep 13 09:27 model_eng
total 0
drwxr-xr-x 1 root root 0 Sep 13 09:27 .
drwxr-xr-x 1 root root 0 Sep 13 08:33 ..
-rw-r--r-- 1 root root 2 Sep 13 09:27 added_tokens.json
-rw-r--r-- 1 root root 473 Sep 13 09:25 config.json
-rw-r--r-- 1 root root 435780426 Sep 13 09:27 pytorch_model.bin
-rw-r--r-- 1 root root 112 Sep 13 09:27 special_tokens_map.json
-rw-r--r-- 1 root root 213450 Sep 13 09:27 vocab.txt

In [0]:
%sh
ls .
ls -la /home/finetune_lm/model/
zip -r /home/finetune_lm/bert_finetuned_model.zip /home/finetune_lm/.
echo
ls -la /home/finetune_lm/

In [0]:
%sh
cd /home/finetune_lm
ls -la .
unzip -t bert_finetuned_model.zip

In [0]:
dbutils.fs.cp("home:/finetune_lm/model/pytorch_model.bin", "home:/finetune_lm/model/")